In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os
import numpy as np
import pandas as pd
import kego.plotting
import polars as pl
import os
import pandas as pd
from glob import glob
from collections import defaultdict
import numpy as np
from tqdm import tqdm
import kego.parallelize

In [ ]:
PATH_DATA = "/mnt/e/ariel-data-challenge-2025"
PATH_DATA_TRAIN = os.path.join(PATH_DATA, "train")

In [ ]:
PLANET_ID = 1010375142
# signal_f = pl.read_parquet("~/Downloads/FGS1_signal_0_1010375142.parquet")
# signal_c = pl.read_parquet("~/Downloads/AIRS-CH0_signal_0_1010375142.parquet")
# train = pl.read_csv("~/Downloads/train.csv")
# train_info = pl.read_csv("~/Downloads/train_star_info.csv")
signal_f = pl.read_parquet(
    f"{PATH_DATA_TRAIN}/{PLANET_ID}/FGS1_signal_0.parquet"
)
signal_c = pl.read_parquet(
    f"{PATH_DATA_TRAIN}/{PLANET_ID}/AIRS-CH0_signal_0.parquet"
)
train = pl.read_csv(f"{PATH_DATA}/train.csv")
wavelengths = pl.read_csv(f"{PATH_DATA}/wavelengths.csv")
train_info = pl.read_csv(f"{PATH_DATA}/train_star_info.csv")

In [ ]:
%%writefile f_read_and_preprocess.py

def process_planet(planet_id, path_planets):
    f_signal = pl.read_parquet(f'{path_planets}/{planet_id}/FGS1_signal_0.parquet')
    mean_signal = f_signal.cast(pl.Int32).sum_horizontal().cast(pl.Float32).to_numpy() / 1024 # mean over the 32*32 pixels
    net_signal = mean_signal[1::2] - mean_signal[0::2]
    return net_signal

def f_read_and_preprocess(dataset, adc_info, planet_ids):
    """Read the FGS1 files for all planet_ids and extract the time series.
    
    Parameters
    dataset: 'train' or 'test'
    adc_info: metadata dataframe, either train_adc_info or test_adc_info
    planet_ids: list of planet ids
    
    Returns
    dataframe with one row per planet_id and 67500 values per row
    
    """
    # f_raw_train = np.full((len(planet_ids), 67500), np.nan, dtype=np.float32)
    path_planets = f"{PATH_DATA}/{dataset}"
    net_signals = kego.parallelize.parallelize(function=process_planet, args_zipped=((planet_id, path_planets) for planet_id in planet_ids))
    return net_signals

In [ ]:
%%time
exec(open("f_read_and_preprocess.py", "r").read())
f_raw_train = f_read_and_preprocess("train", train_info, train["planet_id"])
with open("f_raw_train.pickle", "wb") as f:
    pickle.dump(f_raw_train, f)